<a href="https://colab.research.google.com/github/swarupa1/piya/blob/master/GENAI_CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
#Load Dataset
bank = pd.read_csv("BankFAQs.csv")
bank.head()

,Question,Answer,Class
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security


In [4]:
bank["content"] = bank.apply(lambda row: f"Question: {row['Question']}\nAnswer: {row['Answer']}", axis=1)

In [5]:
bank.head()


,Question,Answer,Class,content
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security,Question: Do I need to enter ‘#’ after keying ...
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security,Question: What details are required when I wan...
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security,Question: How should I get the IVR Password i...
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security,Question: How do I register my Mobile number f...
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security,Question: How can I obtain an IVR Password \nA...


In [ ]:
!pip install langchain


In [6]:
from langchain.docstore.document import Document

# Prepare documents for LangChain
documents = []
for _, row in bank.iterrows():
    documents.append(Document(page_content=row["content"], metadata={"class": row["Class"]}))

In [7]:
documents[1]


Document(metadata={'class': 'security'}, page_content='Question: What details are required when I want to perform a secure IVR transaction\nAnswer: To perform a secure IVR transaction, you will need your 16-digit Card number, Card expiry date, CVV number, mobile number and IVR password.')

In [ ]:
!pip install langchain_community


In [ ]:
!pip install sentence-transformers


Loading Data into Chroma DB¶


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

/tmp/ipython-input-999870367.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hg_embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-999870367.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://

In [ ]:
!pip install chromadb


In [9]:
persist_directory = '/content/my_new_directory'

In [10]:
from langchain.vectorstores import Chroma

#persist_directory = '/content/my_new_directory'

langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="chatbot_finance",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

In [ ]:
!pip install bitsandbytes


In [11]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

model_id = 'HuggingFaceH4/zephyr-7b-beta'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cpu


In [ ]:
!pip install accelerate


In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained(
         model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [1]:
# Initialize the query pipeline with increased max_length
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=6000,  # Increase max_length
    max_new_tokens=500,  # Control the number of new tokens generated
    device_map="auto",
)

NameError: name 'transformers' is not defined

In [5]:
llm = HuggingFacePipeline(pipeline=query_pipeline)


NameError: name 'query_pipeline' is not defined

In [6]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Step 1: Define the Hugging Face pipeline
query_pipeline = pipeline("text-generation", model="gpt2")

# Step 2: Wrap it with LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=query_pipeline)

# Step 3: Ask a question
question = "What is a Chatbot and how is it used in the Finance Domain?"
response = llm(question)

print(response)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-2728241597.py:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipeline)
/tmp/ipython-input-2728241597.py:12: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(question)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is a Chatbot and how is it used in the Finance Domain?

Chatbots are the most versatile and versatile of all the different kinds of chatbots available nowadays. Every single one of them is able to interact with you with a huge range of different kinds of information. And when you try to use a Chatbot to manage information from the past, it will only work for a limited amount of time to get back what you want.

What are the advantages over other types of chatbots?

Chatbots are a great way to connect with people and to connect with your friends and family. They're very social and very simple to use. They're a great way to get information from your friends and family.

Most Chatbots are made up of simple and simple commands, like "help", "info", "help", "info", and "help" You can even use them as a tool to communicate with other chatbots.

You can tell them what you want to say, how much you want to hear, but what to do when you want to talk to them. They will respond to you and rep

In [12]:
from IPython.display import display, Markdown
from langchain_community.llms import HuggingFacePipeline # Import HuggingFacePipeline
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

llm = HuggingFacePipeline(pipeline=query_pipeline)

question = "What is Chatbot and How it used in Finance Domain?"
response = llm(prompt=question)

full_response =  f"Question: {question}\nAnswer: {response}"
display(Markdown(colorize_text(full_response)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




**<font color='red'>Question:</font>** What is Chatbot and How it used in Finance Domain?


**<font color='green'>Answer:</font>** What is Chatbot and How it used in Finance Domain?

Chatbot is a JavaScript library written in Python. It uses JavaScript's built-in JSON API to get the information and messages into a chatbot.

Chatbot's main focus is sending messages to clients and clients. It uses the JSON API to get the information and messages.

Chatbot's main purpose is to be a useful tool for the finance industry. It is not meant to be a replacement for your bank and a payment processor. It is intended to be a tool for those who want to get a feel for the financial market and their market position.

The main goal of Chatbot is to be a useful tool for those interested in financial markets. It is not meant to replace your bank or a payment processor. It is intended to be a tool for those interested in financial markets.

Chatbot is a JavaScript library written in Python. It uses JavaScript's built-in JSON API to get the information and messages into a chatbot.

Chatbot is a JavaScript library written in Python. It uses JavaScript's built-in JSON API to get the information and messages into a chatbot.

Chatbot runs on your computer in an environment that can handle any number of clients and servers. Chatbot works on Windows,

In [3]:
from langchain.llms import HuggingFacePipeline


In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

# Define the prompt template
template = """
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: {question}
Context: {context}
Answer:
"""
PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

retriever = langchain_chroma.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

def chat_with_rag():
    print("Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.")
    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit"]:
            break
        context = "Your context here"  # Provide context if necessary, otherwise leave it empty
        try:
            result = qa_chain({"context": context, "query": query})
            print(f"Chatbot: {result['result']}")
        except RuntimeError as e:
            print(f"RuntimeError encountered: {e}")

# Run the chat
chat_with_rag()

Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: 
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: hi
Context: A chatbot is an AI software that can simulate a conversation. In finance, it is used for customer support, fraud detection, and financial advice.
Answer:
Question: I have a question about the question "What is the best way to get my money back? Is it a coupon?"
Answer:
Question: Thank you for answering.
Context: A bank takes a customer's money and pays its fee.
Answer:
Question: What sort of payment method do you use to make a payment?
Answer:
Question: How do I set up a payment plan?
Answer:
Question: What do you call a payment plan?
Context: A loan is taken out in a bank and then a user can make a payment.
Answer:
Question: I have a question about a loan.
Context: A bank can accept a loan.
Answer:
Question: How does a bank handle a loan?
Answer:
Question: Where can I find an online loan?
Context: A 

In [8]:
pip install langchain transformers chromadb
